This is a python script which is a replicated MapBasic code from a file named **BusLines_Frequency_F6.MBX** 


_Important Note: Python v3.9.1 is needed since it comes compiled with SQLite version 3.33.0. This version of SQLite has introduced some new SQL JOIN statements which are used in the script._


**Create folder structure:**

```
D:\Routelines
        \data
        \db
        \rlc
        \shp

\data -> put here "NOCTable.csv" and CIF files in subfolders by region
\db   -> here will be created sqlite db file
\rlc  -> here will be the files related to "Route Line Creator" (input CSV and output result)
\shp  -> here will be created final shapefile
```

In [1]:
project = "D:/Routelines/"
cif_path = project + 'data/'
db_path = project + 'db/'
rlc_input_path = project + 'rlc/'

RegionID = ['EA'] #,'W','NE','EM','WM','SW','Y','NW','S','SE','L']

In [2]:
import os
import sys
import csv

import io
import psycopg2

import math
import mmap
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

print('Python v' + sys.version)
print('------------------------------------------------------------------------------')
print('Pandas v' + pd.__version__)
print('NumPy v' + np.__version__)

Python v3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
------------------------------------------------------------------------------
Pandas v1.2.4
NumPy v1.19.4


#### global variables definitions

In [3]:
import click
import configparser

# instantiate
config = configparser.ConfigParser()

# parse existing file
config.read('setup.ini')

# read values from a section DATABASE
db_server = config.get('database', 'db_server')
db_port = config.getint('database', 'db_port')
db_def = config.get('database', 'db_def')
db_name = config.get('database', 'db_name')
db_schema = config.get('database', 'db_schema')
db_user = config.get('database', 'db_user')
db_password = config.get('database', 'db_password')

# read values from a section PROJECT
version = config.get('project', 'version')
dir_prj = config.get('project', 'dir_prj')
dir_cif = config.get('project', 'dir_cif')
cif_file = config.get('project', 'cif_file')
pts_table = config.get('project', 'pts_table')

def config_read(ini_filename, section, value):

    config = configparser.ConfigParser()
    config.read(ini_filename)
    
    return config.get(section, value)

def config_print(ini_filename):

    print(95*"-")
    print("Current configuration")
    print("Project data folder   : ", dir_prj)
    print("CIF data folder       : ", dir_cif)
    print("CIF file              : ", cif_file)
    print("PTStops tablename     : ", pts_table)
    print(95*"-")
    print("db server   : ", db_server)
    print("db port     : ", db_port)
    print("db default  : ", db_def)
    print("database    : ", db_name)
    print("schema      : ", db_schema)
    print("user        : ", db_user)
    print("password    : ", db_password)
    print(95*"-")

def config_write(ini_filename):
    config = configparser.ConfigParser()
    config.read(ini_filename)

    print("Set database parameters (ENTER for default values):")
    print(23*"-")
    
    db_server = click.prompt("db_server: ", type=str, default='localhost')
    config.set('database', 'db_server', db_server)

    db_port = click.prompt("db port: ", type=str, default='5434')
    config.set('database', 'db_port', db_port)

    db_def = click.prompt("db def: ", type=str, default='postgres')
    config.set('database', 'db_def', db_def)

    db_name = click.prompt("db name: ", type=str, default='routelinesuk')
    config.set('database', 'db_name', db_name)

    db_schema = click.prompt("db schema: ", type=str, default='rl')
    config.set('database', 'db_schema', db_schema)

    db_user = click.prompt("db user: ", type=str, default='postgres')
    config.set('database', 'db_user', db_user)

    db_password = click.prompt("db password: ", type=str, default='softdesk')
    config.set('database', 'db_password', db_password)

    print("Set project file management parameters:")
    print(38*"-")

    dir_prj = click.prompt("Project data main folder: ", type=str, default='D:/Routelines')
    config.set('project', 'dir_prj', dir_prj)

    dir_cif = click.prompt("CIF data folder: ", type=str, default='D:/Routelines/data/')
    config.set('project', 'dir_cif', dir_cif)

    cif_file = click.prompt("CIF filename: ", type=str, default='Bus_1.cif')
    config.set('project', 'cif_file', cif_file)

    try:
        with open(ini_filename, 'w') as configfile:
            config.write(configfile)
    except IOError:
        print(f"Unable to create {ini_filename} file!")

### Database setup

In [4]:
import sys
from getpass import getpass

import psycopg2

from sqlalchemy import create_engine
from sqlalchemy.schema import CreateSchema
from sqlalchemy_utils import database_exists


def db_connect(db_server, db_port, dbname, db_user, db_password):

    # db_password = getpass(f"Please enter password for user {db_user} @ database {dbname}:")

    # db connection:
    engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_server}:{db_port}/{dbname}")

    if database_exists(engine.url):
        db_status = f"Successfully connected to database '{dbname}' on server '{db_server}' !"
        print(db_status)
        return engine
    else:
        db_status = f"Could not connect to '{dbname}' database on server '{db_server}'!\n"\
                    "Please check if database exist or if configuration parameters are correct."
        print(db_status)
        return 0


def connect(db_name, db_user, db_password, db_server, db_port):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_password, host=db_server, port=db_port)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

#### 1.  CIF data extraction functions

In [5]:
def noc_table(conn, dir_cif):
    NOCTable = pd.read_csv(dir_cif + 'NOCTable.csv')
    NOCTable.to_sql('noctable', conn, schema='rl', if_exists='replace', index = False, chunksize = 100000)
    conn.execute('''CREATE UNIQUE INDEX noccode_idx ON rl.noctable USING btree ("NOCCODE" text_pattern_ops ASC NULLS LAST) TABLESPACE pg_default''')
    conn.execute("COMMIT")

In [6]:
def mapcount(filename):
    print("Counting number of lines...")
    f = open(filename, "r+")
    buf = mmap.mmap(f.fileno(), 0)
    lines = 0
    readline = buf.readline
    while readline():
        lines += 1
    print(f'{lines} lines in the file!')
    return lines

In [7]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [8]:
def insert_with_progress(conn, df, tablename):
    chunksize = int(len(df) / 10) # 10%
    with tqdm(total=len(df)) as pbar:
        for i, cdf in enumerate(chunker(df, chunksize)):
            cdf.to_sql(tablename, conn, schema='rl', if_exists='append', index=False, method='multi') 
            pbar.update(chunksize)
            pbar.set_description(f'Inserting to database...')
    print("Done!")

In [9]:
# Reading CIF file

def read_cif(cif_file):
    print(95*"-")
    print(f'CIF file size is {round(os.stat(cif_file).st_size / (1024 * 1024), 2)} MB')   
    count_lines = mapcount(cif_file)
    chunk_size=1000000
    chunks = []
    loops = math.ceil(count_lines/chunk_size)
    i=0
    with tqdm(total = loops, file = sys.stdout) as pbar:
        reader = pd.read_csv(cif_file, names=['CODE'], header=None, sep='!', iterator=True)
        while i <= loops:
            try:
                i+=1
                chunk = reader.get_chunk(chunk_size)
                chunks.append(chunk)
                pbar.set_description('Importing CIF data to Pandas dataframe')
                pbar.update(1)
            except StopIteration:
                loop = False
                cif_data = pd.concat(chunks, ignore_index=True)
                pbar.update(1)
    pbar.close()
    print("CIF data imported!")
    return cif_data

In [10]:
# PTStops table creation

def GetPtStops(conn, cif_data, tablename):
    print("Extracting QB nodes in progress...")
    QB = cif_data['CODE'].str.extract('(^QB.*)').dropna()
    QB.columns = ['CODE']
    print("QB nodes extracted:", len(QB))
    COLUMN_NAMES = ['naptanid','xcoord','ycoord']
    PTStops = pd.DataFrame(columns=COLUMN_NAMES)
    PTStops['naptanid'] = QB['CODE'].str.slice(start=3, stop=15).str.rstrip()
    PTStops['xcoord'] = QB['CODE'].str.slice(start=15, stop=21)
    PTStops['ycoord'] = QB['CODE'].str.slice(start=23, stop=29)
    # convert columns "xcoord" and "ycoord" to numeric
    PTStops[["xcoord", "ycoord"]] = PTStops[["xcoord", "ycoord"]].apply(pd.to_numeric, downcast='integer')
    insert_with_progress(conn, PTStops, tablename)
    conn.execute('''CREATE INDEX ptstops_idx ON rl.ptstops USING btree (naptanid text_pattern_ops ASC NULLS LAST) TABLESPACE pg_default''')
    conn.execute("COMMIT")
    return PTStops  

#### Routes extraction

In [11]:
def copy_from_stringio_progress(conn, df, table, cols):
    """
    Here we are going save the dataframe in memory and use copy_from() to copy it to the table
    """
    # save dataframe to an in memory buffer
    buffer = io.StringIO()
    df.to_csv(buffer, index_label='id', header=False)
    buffer.seek(0)

    cursor = conn.cursor()

    count_lines=df.shape[0]
    chunk_size=1000000
    chunks = []
    loops = math.ceil(count_lines/chunk_size)
    i=0

    with tqdm(total = loops, file = sys.stdout) as pbar:
        while i <= loops:
            try:
                i+=1
                reader = cursor.copy_from(buffer, table, sep=",", columns=cols )
                conn.commit()
                pbar.set_description('Uploading to PostgreSQL...')
                pbar.update(1)
            except (Exception, psycopg2.DatabaseError) as error:
                print("Error: %s" % error)
                conn.rollback()
                cursor.close()
                return 1
    pbar.close()
    print(f"Uploading '{table}' to PostgreSQL done!")
    cursor.close()


In [12]:
def copy_from_stringio(conn, df, table, cols):
    """
    Here we are going save the dataframe in memory and use copy_from() to copy it to the table
    """
    # save dataframe to an in memory buffer
    buffer = io.StringIO()
    df.to_csv(buffer, index_label='id', header=False)
    buffer.seek(0)
    
    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",", columns=cols )
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print(f"Uploading '{table}' to PostgreSQL done!")
    cursor.close()

In [13]:
def GetRouteDataToSQL(conn, conn2, cif_df, region):
    
    # temp list for RouteLines1 columns

    l_Region = []
    l_BMRouteID_1 = []
    l_BM_StartStopID_1 = []
    l_AnodeStopID = []
    l_BnodeStopID = []
    l_AnodeXcoord = []
    l_AnodeYcoord = []
    l_BnodeXcoord = []
    l_BnodeYcoord = []
    # l_id_1 = []

    # temp list for MainTable columns

    l_OperatorCode = []
    l_ServiceNum = []
    l_BM_RouteID_2 = []
    l_BM_StartStopID_2 = []
    l_Direction = []
    l_StopID = []
    l_DeptTime = []
    l_Seq = []
    l_Mon = []
    l_Tue = []
    l_Wed = []
    l_Thu = []
    l_Fri = []
    l_Sat = []
    l_Sun = []
    l_TotalWeekly = []
    # l_id_2 = []

    # routelines1 to PostgreSQL

    conn.execute('''DROP TABLE IF EXISTS rl.routelines1''')
    conn.execute("COMMIT")
    conn.execute('''CREATE TABLE rl.routelines1 ( id integer PRIMARY KEY,
                                                  region text,
                                                  bmrouteid text,
                                                  bm_startstopid text,
                                                  anodestopid text,
                                                  bnodestopid text,
                                                  anodexcoord text,
                                                  anodeycoord text,
                                                  bnodexcoord text,
                                                  bnodeycoord text) TABLESPACE pg_default'''
                                                  )
    conn.execute("COMMIT")
    conn.execute('ALTER TABLE rl.routelines1 OWNER to postgres')
    conn.execute("COMMIT")
    # conn.execute('''CREATE INDEX rl1_bm_startstopid_idx ON rl.routelines1 USING btree (bm_startstopid text_pattern_ops ASC NULLS LAST) TABLESPACE pg_default''')
    # conn.execute("COMMIT")
    # conn.execute('''CREATE INDEX rl1_bmrouteid_idx ON rl.routelines1 USING btree (bmrouteid text_pattern_ops ASC NULLS LAST) TABLESPACE pg_default''')
    # conn.execute("COMMIT")

    # maintable to PostgreSQL

    conn.execute('''DROP TABLE IF EXISTS rl.maintable''')
    conn.execute("COMMIT")    
    conn.execute('''CREATE TABLE rl.maintable( id integer PRIMARY KEY,
                                               operatorcode text,
                                               servicenum text,
                                               bm_routeid text,
                                               bm_startstopid text,
                                               direction text,
                                               stopid text,
                                               depttime integer,
                                               seq integer,
                                               mon integer,
                                               tue integer,
                                               wed integer,
                                               thur integer,
                                               fri integer,
                                               sat integer,
                                               sun integer,
                                               totalweekly integer) TABLESPACE pg_default'''
                                               )
    conn.execute("COMMIT")
    conn.execute('ALTER TABLE rl.maintable OWNER to postgres')
    conn.execute("COMMIT")
    # conn.execute('''CREATE INDEX mt_bm_startstopid_idx ON rl.maintable USING btree (bm_startstopid text_pattern_ops ASC NULLS LAST)  TABLESPACE pg_default''')
    # conn.execute("COMMIT")    
    # conn.execute('''CREATE INDEX mt_bm_routeid_idx ON rl.maintable USING btree (bm_routeid text_pattern_ops ASC NULLS LAST) TABLESPACE pg_default''')
    # conn.execute("COMMIT")    
#   -----------
    print("Data extraction from CIF to lists...")
    
    BankHolOnly = "" 

    for x in cif_df['CODE']:
        if x[0:2]=='QS':
    #         -----------------------
    #         QS
    #         -----------------------
            SeqNo = 0
            AnodeStr = "XXXX"
            RouteIDFull = x[38:65]
    #         print("1-",RouteIDFull)
            RouteIDV = RouteIDFull[:4].rstrip()
    #         print("2-",RouteIDV)
            RouteIDFull = x[37:65]
    #         print("3-",RouteIDFull)        
            BankHolOnly = RouteIDFull[0]
    #         print("4-",BankHolOnly)
            if BankHolOnly != "":
                pass
            RouteIDFull =x[3:]
    #         print("5-"+RouteIDFull)                
            RouteOpV = RouteIDFull[:4].rstrip()
    #         print("6-"+RouteOpV)
            ServiceIDv = RouteIDV
            RouteIDFull = x[64:]
    #         print("7-"+RouteIDFull)        
            DirectionV = RouteIDFull[0]
    #       RouteIDV = region + "_" + RouteOpV + "_" +  RouteIDV + "_" + DirectionV
            RouteIDV = RouteOpV + "_" +  RouteIDV + "_" + DirectionV
    #         print("8-"+RouteIDV)
    #         RouteIDFull = x[29:]
            MonV = int(x[29:][0])
    #         print("MonV =", MonV)
            TueV = int(x[30:][0])
    #         print("TueV =", TueV)
            WedV = int(x[31:][0])
    #         print("WedV =", WedV)
            ThuV = int(x[32:][0])
    #         print("ThuV =", ThuV)
            FriV = int(x[33:][0])
    #         print("FriV =", FriV)
            SatV = int(x[34:][0])
    #         print("SatV =", SatV)
            SunV = int(x[35:][0])
    #         print("SunV =", SunV)
    #         print("------------")
        if BankHolOnly == "B":
            BankLoop = BankLoop + 1
            if BankLoop == 1:
                print("WARNING: Bank holiday services exist!")
        else:
            if x[0:2] in ('QO','QI','QT'):
    #             print(x[0:2])
    #             -----------------------
    #             QO / QI / QT
    #             -----------------------
                RouteIDFull = x[2:]
    #             print("9 -", RouteIDFull)
                RSStopID = RouteIDFull[:12].rstrip()
    #             print("10-", RSStopID)
                StopArea = RouteIDFull[:4].rstrip()
                BnodeStr = RSStopID
    #             print("StopArea -",StopArea)
    #             print("BnodeStr -",BnodeStr)
                RouteIDFull = x[14:]
    #             print("11-", RouteIDFull)
                TimeNum = RouteIDFull[:4]
    #             print("12-", TimeNum)
                SeqNo = SeqNo + 1
                if SeqNo == 1:
                    StartStopID = RSStopID
                if SeqNo > 1:
                    # print("RouteLines1:")
                    l_Region.append(region)
                    l_BMRouteID_1.append(RouteIDV)
                    l_BM_StartStopID_1.append(StartStopID)
                    l_AnodeStopID.append(AnodeStr)
                    l_BnodeStopID.append(BnodeStr)
                    l_AnodeXcoord.append(0)
                    l_AnodeYcoord.append(0)
                    l_BnodeXcoord.append(0)
                    l_BnodeYcoord.append(0)
                    # l_id_1.append(SeqNo)
                AnodeStr = RSStopID
                if int(TimeNum) >= 0 and int(TimeNum) < 2400:
                    # print("MainTable:")
                    l_OperatorCode.append(RouteOpV)
                    l_ServiceNum.append(ServiceIDv)
                    l_BM_RouteID_2.append(RouteIDV)
                    l_BM_StartStopID_2.append(StartStopID)
                    l_Direction.append(DirectionV)
                    l_StopID.append(RSStopID)
                    l_DeptTime.append(TimeNum)
                    l_Seq.append(SeqNo)
                    l_Mon.append(MonV)
                    l_Tue.append(TueV)
                    l_Wed.append(WedV)
                    l_Thu.append(ThuV)
                    l_Fri.append(FriV)
                    l_Sat.append(SatV)
                    l_Sun.append(SunV)
                    l_TotalWeekly.append(0) 
                    # l_id_2.append(SeqNo)
    
#   -----------
# Pandas dataframe creations

    # - RouteLines1
    print("Loading RouteLines1 to Pandas dataframe...")
    
    RouteLines1 = pd.DataFrame(
        {'region':l_Region,
         'bmrouteid':l_BMRouteID_1,
         'bm_startstopid':l_BM_StartStopID_1,
         'anodestopid':l_AnodeStopID,
         'bnodestopid':l_BnodeStopID,
         'anodexcoord':l_AnodeXcoord ,
         'anodeycoord':l_AnodeYcoord ,
         'bnodexcoord':l_BnodeXcoord ,
         'bnodeycoord':l_BnodeYcoord
        #  ,
        #  'id':l_id_1
        })
    

    # - MainTable
    print("Loading MainTable to Pandas dataframe...")
    
    MainTable = pd.DataFrame(
        {'operatorcode':l_OperatorCode,
         'servicenum':l_ServiceNum,
         'bm_routeid':l_BM_RouteID_2,
         'bm_startstopid':l_BM_StartStopID_2,
         'direction':l_Direction,
         'stopid':l_StopID,
         'depttime':l_DeptTime,
         'seq':l_Seq,
         'mon':l_Mon,
         'tue':l_Tue,
         'wed':l_Wed,
         'thur':l_Thu,
         'fri':l_Fri,
         'sat':l_Sat,
         'sun':l_Sun,
         'totalweekly':l_TotalWeekly
        #  ,
        #  'id':l_id_2
        })
    
    print(f"RouteLines1 size: {len(RouteLines1.index)} rows")
    print(f"MainTable size:   {len(MainTable.index)} rows")

    # ------------------------------------------------------------------------------------------------------------
    # EMPTY temp list for RouteLines1 columns

    l_Region = []
    l_BMRouteID_1 = []
    l_BM_StartStopID_1 = []
    l_AnodeStopID = []
    l_BnodeStopID = []
    l_AnodeXcoord = []
    l_AnodeYcoord = []
    l_BnodeXcoord = []
    l_BnodeYcoord = []
    # l_id_1 = []

    # EMPTY temp list for MainTable columns

    l_OperatorCode = []
    l_ServiceNum = []
    l_BM_RouteID_2 = []
    l_BM_StartStopID_2 = []
    l_Direction = []
    l_StopID = []
    l_DeptTime = []
    l_Seq = []
    l_Mon = []
    l_Tue = []
    l_Wed = []
    l_Thu = []
    l_Fri = []
    l_Sat = []
    l_Sun = []
    l_TotalWeekly = []    
    # l_id_2 = []

    # ------------------------------------------------------------------------------------------------------------
    #  Upload to db

    print("Uploading RouteLines1 df to PostgreSQL...")    
    # RouteLines1.to_sql('routelines1', conn, schema='rl' ,if_exists='append', method='multi', index = False) # SLOW!
    copy_from_stringio_progress(conn2, RouteLines1, 'rl.routelines1', ('id','region', 'bmrouteid', 'bm_startstopid', 'anodestopid', 'bnodestopid', 'anodexcoord', 'anodeycoord', 'bnodexcoord', 'bnodeycoord'))
        
    print("Uploading MainTable df to PostgreSQL...")    
    # MainTable.to_sql('maintable', conn, schema='rl' ,if_exists='append', method='multi', index = False) # SLOW!
    copy_from_stringio_progress(conn2, MainTable, 'rl.maintable', ('id','operatorcode','servicenum','bm_routeid','bm_startstopid','direction','stopid','depttime','seq','mon','tue','wed','thur','fri','sat','sun','totalweekly'))

# ------------------------------------------------------------------------------------------------------------

In [14]:
def ptstops_update(conn):

    print("Updating 'PTStops_temp'...")
    
    conn.execute('''INSERT INTO rl.ptstops_temp (naptanid, xcoord, ycoord) SELECT naptanid, xcoord, ycoord FROM rl.ptstops GROUP BY naptanid, xcoord, ycoord''')    
    conn.execute("COMMIT")
    conn.execute('''CREATE INDEX ptstops_tmp_idx ON rl.ptstops_temp USING btree (naptanid text_pattern_ops ASC NULLS LAST) TABLESPACE pg_default''')
    conn.execute("COMMIT")

In [15]:
def rl1_update(conn):    

    print("Updating RouteLines1 RouteID ...")
    
    conn.execute('''UPDATE rl.routelines1 
                    SET bmrouteid = bmrouteid || '_' || LEFT(tmp.xcoord,3) || '_' || LEFT(tmp.ycoord,3) 
                    FROM rl.ptstops_temp tmp
                    WHERE bm_startstopid = tmp.naptanid''')
    conn.execute("COMMIT")

In [16]:
def mt_update(conn):    

    print("Updating Maintable RouteID...")
    
    conn.execute('''UPDATE rl.maintable 
                    SET bm_routeid = bm_routeid || '_' || LEFT(tmp.xcoord,3) || '_' || LEFT(tmp.ycoord,3) 
                    FROM rl.ptstops_temp tmp
                    WHERE bm_startstopid = tmp.naptanid''')
    conn.execute("COMMIT")

#### 2. CreateFreq

In [17]:
def TempGetFreq(conn, ColName, depttime1, depttime2):
    conn.execute('''CREATE TABLE rl.tempgetfreq AS SELECT * FROM rl.maintable WHERE ''' + ColName + ''' = 1 AND depttime >= ''' + str(depttime1) + ''' AND depttime < ''' + str(depttime2) + ''' AND seq = 1''')
    conn.execute("COMMIT")
    conn.execute('''CREATE TABLE rl.sumfreq AS SELECT bm_routeid, SUM(seq) AS sumfreq FROM rl.tempgetfreq GROUP BY bm_routeid''')
    conn.execute("COMMIT")
    
def UpdFreqVal(conn, UpdCol, round_value):
    conn.execute('''UPDATE rl.rlfreq SET ''' + UpdCol + ''' = updfreqval.sumfreq/''' + str(round_value) + '''
                    FROM (SELECT rl.rlfreq.bm_routeid, operatorcode, servicenum, direction, operatorname,
                                monearly, monam, monbp, monep, monop, monnight,
                                tueearly, tueam, tuebp, tueep, tueop, tuenight, 
                                wedearly, wedam, wedbp, wedep, wedop, wednight,
                                thurearly, thuram, thurbp, thurep, thurop, thurnight,
                                friearly, friam, fribp, friep, friop, frinight,
                                satearly, satam, satbp, satep, satop, satnight, 
                                sunearly, sunam, sunbp, sunep, sunop, sunnight, 
                                id, sumfreq 
                        FROM rl.rlfreq, rl.sumfreq 
                        WHERE rl.rlfreq.bm_routeid = sumfreq.bm_routeid) AS updfreqval 
                    WHERE rl.rlfreq.id = updfreqval.id''')
    conn.execute("COMMIT")
    conn.execute('''UPDATE rl.rlfreq SET ''' + UpdCol + ''' = 0 WHERE ''' + UpdCol + ''' is NULL''')
    conn.execute("COMMIT")
    conn.execute('''UPDATE rl.rlfreq SET ''' + UpdCol + ''' = ROUND(CAST(''' + UpdCol + ''' AS numeric), 2)''')
    conn.execute("COMMIT")
    conn.execute('''DROP TABLE IF EXISTS rl.tempgetfreq''')
    conn.execute("COMMIT")
    conn.execute('''DROP TABLE IF EXISTS rl.sumfreq''')
    conn.execute("COMMIT")

def CreateFreq(conn):
    
    ColNames = {1:'mon', 2:'tue', 3:'wed', 4:'thur', 5:'fri', 6:'sat', 7:'sun'}
    
    DepTimes = {
                'am':[2.0, 700,900],
                'bp':[7.0, 900,1600],
                'ep':[2.0, 1600,1800],
                'op':[6.0, 1800,2400],
                'night':[3.0, 0,300],
                'early':[3.0, 400,700]
               }    
    conn.execute('''DROP TABLE IF EXISTS rl.rlfreq''')    
    conn.execute("COMMIT")
    conn.execute('''CREATE TABLE rl.rlfreq(
       bm_routeid    text,
       operatorcode  text,
       servicenum    text,
       direction     text,
       operatorname  text,
       monearly      real,
       monam         real,
       monbp         real,
       monep         real,
       monop         real,
       monnight      real,
       tueearly      real,
       tueam         real,
       tuebp         real,
       tueep         real,
       tueop         real,
       tuenight      real,
       wedearly      real,
       wedam         real,
       wedbp         real,
       wedep         real,
       wedop         real,
       wednight      real,
       thurearly     real,
       thuram        real,
       thurbp        real,
       thurep        real,
       thurop        real,
       thurnight     real,
       friearly      real,
       friam         real,
       fribp         real,
       friep         real,
       friop         real,
       frinight      real,
       satearly      real,
       satam         real,
       satbp         real,
       satep         real,
       satop         real,
       satnight      real,
       sunearly      real,
       sunam         real,
       sunbp         real,
       sunep         real,
       sunop         real,
       sunnight      real,
       id serial PRIMARY KEY) TABLESPACE pg_default''')
    
    conn.execute('''
                    INSERT INTO rl.rlfreq (bm_routeid, operatorcode, servicenum, direction)
                    SELECT bm_routeid, operatorcode, servicenum, direction
                    FROM rl.maintable 
                    GROUP BY bm_routeid, operatorcode, servicenum, direction 
                    ORDER BY bm_routeid
                    ''')
    conn.execute("COMMIT")

    conn.execute('''UPDATE rl.rlfreq 
                    SET operatorname = "OperatorPublicName"
                    FROM (SELECT * From rl.rlfreq, rl.noctable WHERE operatorcode = "NOCCODE" AND NOT operatorcode = '') AS updop
                    WHERE updop.id = rl.rlfreq.id''')
    conn.execute("COMMIT")
    
    for key1, value1 in ColNames.items():
        print("Freq. Calculations for " + value1 + "...")
        for key2, value2 in DepTimes.items():
    #       print(key1, "ColName=" + str(value1+key2), value2[0],value2[1],value2[2])
    #       print("TempGetFreq", str(value1), value2[1], value2[2])
            TempGetFreq(conn, str(value1), value2[1], value2[2])
    #       print("UpdFreqVal", str(value1+key2), value2[0])
            UpdFreqVal(conn, str(value1+key2), value2[0])    


#### 3. ABSorter

In [18]:
def ABsorter(conn):
    
    print("Creation of ABNodes and updating...")
    
    conn.execute('''DROP TABLE IF EXISTS rl.abnodes;''')
    conn.execute("COMMIT")
    conn.execute('''CREATE TABLE rl.abnodes(
                                            anodestopid text ,
                                            bnodestopid text ,
                                            anodexcoord integer,
                                            anodeycoord integer,
                                            bnodexcoord integer,
                                            bnodeycoord integer,
                                            id serial) TABLESPACE pg_default''')
    conn.execute("COMMIT")                                            
    conn.execute('''ALTER TABLE rl.abnodes OWNER to postgres''')
    conn.execute("COMMIT")
    conn.execute('''INSERT INTO rl.abnodes (anodestopid, bnodestopid)
                    SELECT AnodeStopID, BnodeStopID
                    FROM rl.routelines1 
                    GROUP BY  AnodeStopID, BnodeStopID''')
    conn.execute("COMMIT")
    conn.execute('''UPDATE rl.abnodes
                    SET anodexcoord = CAST(updanode.xcoord AS integer), anodeycoord = CAST(updanode.ycoord AS integer)
                    FROM (SELECT * FROM rl.abnodes, rl.ptstops_temp WHERE anodestopid = naptanid) AS updanode
                    WHERE updanode.id = abnodes.id''')
    conn.execute("COMMIT")
    conn.execute('''UPDATE rl.abnodes
                    SET bnodexcoord = CAST(updbnode.xcoord AS integer), bnodeycoord = CAST(updbnode.ycoord AS integer)
                    FROM (SELECT * FROM rl.abnodes, rl.ptstops_temp WHERE bnodestopid = naptanid) AS updbnode
                    WHERE updbnode.id = abnodes.id''')
    conn.execute("COMMIT")

In [19]:
from datetime import datetime

def timestamp():
    time = datetime.now()
    time_hms = time.strftime("%H:%M:%S")
    return [time, time_hms]

#### Data extraction from CIF files:

In [20]:
config_print('setup.ini')

-----------------------------------------------------------------------------------------------
Current configuration
Project data folder   :  D:/Routelines
CIF data folder       :  D:/Routelines/data/
CIF file              :  ATCO_Bus.cif
PTStops tablename     :  ptstops
-----------------------------------------------------------------------------------------------
db server   :  localhost
db port     :  5434
db default  :  postgres
database    :  routelinesuk
schema      :  rl
user        :  postgres
password    :  softdesk
-----------------------------------------------------------------------------------------------


In [21]:
def cif():
    print(95*"-")
    region ="UK"
    t1, tp1 = timestamp()[0], timestamp()[1]
    print(f"Processing CIF file [{dir_cif+cif_file}] started at {tp1}")
    
    cif_df = read_cif(dir_cif+cif_file)
    t2, tp2 = timestamp()[0], timestamp()[1]
    print(f"Finished in {round((t2 - t1).total_seconds(),1)} seconds.")
    print("---")
    
    print("Extracting PTStops (QB) data:")
    GetPtStops(conn, cif_df, pts_table)
    t3, tp3 = timestamp()[0], timestamp()[1]
    print(f"Finished in {round((t3 - t2).total_seconds(),1)} seconds.")
    print("---")
    
    GetRouteDataToSQL(conn, conn2, cif_df, region)
    t4, tp4 = timestamp()[0], timestamp()[1]
    print(f"Finished in {round((t4 - t3).total_seconds(),1)} seconds.")
    print("---")

    ptstops_update(conn)
    rl1_update(conn)
    mt_update(conn)
    
    print(f"Frequency calculating:")
    CreateFreq(conn)
    ABsorter(conn)    
    
    print("Creation of 'ABNodesUK', 'RLFreqUK', 'RoutelinesUK'...")
    conn.execute('''CREATE TABLE rl.abnodesuk AS SELECT * FROM rl.abnodes''')
    conn.execute("COMMIT")  
    conn.execute('''CREATE TABLE rl.rlfrequk AS SELECT * FROM rl.rlfreq''')
    conn.execute("COMMIT")  
    conn.execute('''CREATE TABLE rl.routelinesuk AS SELECT bmrouteid, anodestopid, bnodestopid FROM rl.routelines1 GROUP BY bmrouteid, anodestopid, bnodestopid''')
    conn.execute("COMMIT")     

In [22]:
conn = db_connect(db_server, db_port, db_name, db_user, db_password)
conn2 = connect(db_name, db_user, db_password, db_server, db_port)
noc_table(conn, dir_cif)
conn.execute('CREATE TABLE rl.ptstops_temp(naptanid text, xcoord text, ycoord text)')
conn.execute("COMMIT")
cif()

Successfully connected to database 'routelinesuk' on server 'localhost' !
Connecting to the PostgreSQL database...
Connection successful
-----------------------------------------------------------------------------------------------
Processing CIF file [D:/Routelines/data/ATCO_Bus.cif] started at 13:30:38
-----------------------------------------------------------------------------------------------
CIF file size is 13.08 MB
Counting number of lines...
390975 lines in the file!


  0%|          | 0/1 [00:00<?, ?it/s]

CIF data imported!
Finished in 0.3 seconds.
---
Extracting PTStops (QB) data:
Extracting QB nodes in progress...
QB nodes extracted: 8306


  0%|          | 0/8306 [00:00<?, ?it/s]

Done!
Finished in 0.8 seconds.
---
Data extraction from CIF to lists...
Loading RouteLines1 to Pandas dataframe...
Loading MainTable to Pandas dataframe...
RouteLines1 size: 346526 rows
MainTable size:   359128 rows
Uploading RouteLines1 df to PostgreSQL...


  0%|          | 0/1 [00:00<?, ?it/s]

Uploading 'rl.routelines1' to PostgreSQL done!
Uploading MainTable df to PostgreSQL...


  0%|          | 0/1 [00:00<?, ?it/s]

Uploading 'rl.maintable' to PostgreSQL done!
Finished in 6.5 seconds.
---
Updating 'PTStops_temp'...
Updating RouteLines1 RouteID ...
Updating Maintable RouteID...
Frequency calculating:
Freq. Calculations for mon...
Freq. Calculations for tue...
Freq. Calculations for wed...
Freq. Calculations for thur...
Freq. Calculations for fri...
Freq. Calculations for sat...
Freq. Calculations for sun...
Creation of ABNodes and updating...
Creation of 'ABNodesUK', 'RLFreqUK', 'RoutelinesUK'...


<font color='red' style="font-size:30px"><b>Done!</b></font> 

In [23]:

# MapBasic EA region calculations     : 03m:24sec
# Python+SQLite EA region calculations: 00m:10sec  ~20x faster

# MapBasic All regions calculations     : 4h:45m:00sec
# Python+SQLite All regions calculations: 0h:17m:10sec  ~17x faster